# Amplifier Design

We are going to calculate the resistor values for the mid-band amplifier shown below.

![Amplifier Circuit Diagram](circuit.jpg)

Parameters: 
$ V_\mathrm{CC} = 15  \ \mathrm{V},\quad R_\mathrm{S} = 8 \ \mathrm{k\Omega},\quad R_L= 300 \ \mathrm{\Omega} $

All the capacitances: $20 \mathrm{\mu F}$

Transistor parameters of Q1 and Q2: 
$ V_\mathrm{BE} = 0.7 \ \mathrm{V},\quad  \beta = 150,\quad V_\mathrm{A} = 65 \ \mathrm{V} $

Transistor parameters of M3: 
$ K_\mathrm{n} = 5 \ \mathrm{mA/V^2},\quad V_\mathrm{TN} = -2 \ \mathrm{V},\quad λ=0.01 \ \mathrm{V^{−1}} $

In [ ]:
import sympy as sym
from IPython.display import display
sym.init_printing()

In [ ]:
# Variable
v_S = sym.Symbol('v_S')

# Values to decide
R_11, R_12, R_21, R_22, R_31, R_32 = sym.symbols('R_11, R_12, R_21, R_22, R_31, R_32', positive=True)
R_C1, R_E1, R_C2, R_E2, R_S3 = sym.symbols('R_C1, R_E1, R_C2, R_E2, R_S3', positive=True)

# Value to calculate
I_D = sym.symbols('I_D')

# Constants
V_CC, R_S, R_L, V_BE, beta, V_A, V_T, K_n, V_TN, lambda_ = sym.symbols('V_CC, R_S, R_L, V_BE, \\beta, V_A, V_T, K_n, V_TN, \lambda')

In [ ]:
# Design specifications
V_CC = 15  # [V]
R_S = 8 * (10 ** 3)  # [Ohm]
R_L = 300  # [Ohm]

# Q1 and Q2
V_BE = 0.7  # [V]
beta = 150 
V_A = 65  # [V]
V_T = 26 * (10 ** -3)  # [V]

# M3
K_n = 5 * (10 ** -3)  # [A/V^2] 
V_TN = -2  #[V]
lambda_ = 0.01  # [V^−1]

# Designs
Manipulate these values.

In [ ]:
R_11 = 100 * (10 ** 3)  # Ohm
R_12 = 100 * (10 ** 3)  # Ohm
R_C1 = 5 * (10 ** 3)  # Ohm
R_E1 = 5 * (10 ** 3)  # Ohm

R_21 = 100 * (10 ** 3)  # Ohm
R_22 = 100 * (10 ** 3)  # Ohm
R_C2 = 5 * (10 ** 3)  # Ohm
R_E2 = 5 * (10 ** 3)  # Ohm

R_31 = 100 * (10 ** 3)  # Ohm
R_32 = 700 * (10 ** 3)  # Ohm
R_S3 = 1 * (10 ** 3)  # Ohm

In [ ]:
def par(*resistances: int):
    '''
    Calculates the total resistance of parallel resistors.
    '''
    total_recip = 0
    for res in resistances:
        recip = 1 / res
        total_recip = sym.Add(total_recip, recip)
    return 1 / total_recip


# DC Analysis

## Q1

In [ ]:
R_Th1 = par(R_11, R_12)
V_Th1 = R_12 / (R_11 + R_12) * V_CC

R_Th1, V_Th1

In [ ]:
I_B1 = (V_Th1 - V_BE) / (R_Th1 + (beta + 1) * R_E1)
r_pi1 = V_T / I_B1

g_m1 = beta * I_B1 / V_T

r_01 = V_A / (beta * I_B1)

r_pi1, g_m1, r_01

## Q2

In [ ]:
R_Th2 = par(R_21, R_22)
V_Th2 = R_22 / (R_21 + R_22) * V_CC

R_Th2, V_Th2

In [ ]:
I_B2 = (V_Th2 - V_BE) / (R_Th2 + (beta + 1) * R_E2)
r_pi2 = V_T / I_B2

g_m2 = beta * I_B2 / V_T

r_02 = V_A / (beta * I_B2)

r_pi2, g_m2, r_02

## M3

In [ ]:
V_G = R_32 / (R_31 + R_32) * V_CC
V_GS = V_G - I_D * R_S3

Calculation of $I_D$ depends on resistor values.

In [ ]:
eq_M3 = I_D - K_n * (V_GS - V_TN) ** 2  # I_D = K_n * (V_GS - V_TN) ** 2

I_D_ = sym.solve(eq_M3, I_D)

I_D_

In [ ]:
V_GS_ = []
for i in I_D_:
    V_GS_.append(V_G - i * R_S3)

V_GS_

In [ ]:
comparators = []

for j in V_GS_:
    comparators.append(sym.GreaterThan(j, V_TN))

print(comparators)

which = comparators.index(True)
I_D = I_D_[which]
V_GS = V_GS_[which]

I_D, V_GS

In [ ]:
g_m3 = 2 * K_n * (V_GS - V_TN)
r_03 = 1 / (lambda_ * I_D)

g_m3, r_03

# Small Signal Equivalent Circuit

![Small Signal Equivalent Circuit](ac_circuit.jpg)

## Q1

In [ ]:
v_pi1 = par(R_11, R_12, r_pi1) / (par(R_11, R_12, r_pi1) + R_S) * v_S
v_pi2 = - par(r_01, R_C1, R_21, R_22, r_pi2) * g_m1 * v_pi1
A_v1 = v_pi2 / v_S

A_v1, v_pi2 / v_pi1

## Q2

In [ ]:
v_g = - par(r_02, R_C2, R_31, R_32) * g_m2 * v_pi2
A_v2 = v_g / v_pi2

A_v2

## M3

In [ ]:
v_s = sym.symbols('v_s')
eq_ac3 = v_s - par(r_03, R_S3, R_L) * g_m3 * (v_g - v_s)
v_s = sym.solve(eq_ac3, v_s)

if len(v_s) == 1:
    v_s = v_s[0]
    A_v3 = v_s / v_g
    display(A_v3)
else:
    print('The values of v_s are '), v_s

In [ ]:
A_v = A_v1 * A_v2 * A_v3

A_v, v_s / v_S

In [ ]:
eq_g1 = A_v1 + 4
eq_g2 = A_v2 + 4
eq_g3 = A_v3 - 0.8



# LTSpice

Calculating $V_{BE}$

In [ ]:
k = 1.38 * 10 ** -23  # (Boltzmann constant)
T = 293  # (temperature in Kelvin)
q = 1.60 * 10 ** -19  # (electron charge)

I_E1 = (beta + 1) * I_B1
I_S1 = I_E1 * (sym.exp(-q * V_BE/(k * T)))
I_E2 = (beta + 1) * I_B2
I_S2 = I_E2 * (sym.exp(-q * V_BE/(k * T)))

I_S1, I_S2

In [ ]:
I_C1 = beta * I_B1

V_B1 = V_Th1 - R_Th1 * I_B1
V_C1 = V_CC - R_C1 * I_C1
V_E1 = R_E1 * I_E1

V_B1, V_C1, V_E1, I_B1, I_C1, I_E1


In [ ]:
I_C2 = beta * I_B2

V_B2 = V_Th2 - R_Th2 * I_B2
V_C2 = V_CC - R_C2 * I_C2
V_E2 = R_E2 * I_E2

V_B2, V_C2, V_E2, I_B2, I_C2, I_E2

In [ ]:
V_sat = V_GS - V_TN

V_sat